In [9]:
import pulp

# params
slots  = 5
people = ["Spongebob", "Squidward", "Mr. Crabs", "Pearl"]
roles  = ["Fry Cook", "Cashier", "Money Fondler"]
leave  = {"Mr. Crabs": [0,2,3,4]}
max_assignments_per_person = 5

# Create the 'problem'
problem = pulp.LpProblem("rota generator", sense=pulp.LpMaximize)

# Create variables
assignments = pulp.LpVariable.dicts("A", ((s, p, r) for s in range(slots) for p in people for r in roles), cat="Binary")
is_assigned = pulp.LpVariable.dicts("X", people, cat="Binary")

# Add constraints
for slot in range(slots):
    for role in roles:
        # In every time slot, each role is assigned to exactly one person
        problem += pulp.lpSum(assignments[slot, person, role] for person in people) == 1

    for person in people:
        # Nobody is assigned multiple roles in the same time slot
        problem += pulp.lpSum(assignments[slot, person, role] for role in roles) <= 1

for person, bad_slots in leave.items():
    for slot in bad_slots:
        for role in roles:
            # Nobody is assigned a role in a slot they are on leave for
            problem += assignments[slot, person, role] == 0

for person in people:
    # Nobody works too many shifts
    problem += pulp.lpSum(assignments[slot, person, role] for slot in range(slots) for role in roles) <= max_assignments_per_person

# Constrain 'is_assigned' auxiliary variable
for slot in range(slots):
    for person in people:
        for role in roles:
            # If
            problem += is_assigned[person] >= assignments[slot, person, role]

for person in people:
    # Only if
    problem += is_assigned[person] <= pulp.lpSum(assignments[slot, person, role] for slot in range(slots) for role in roles)

# Add objective
problem += pulp.lpSum(is_assigned[person] for person in people)

# Solve with the Coin/Cbc solver
problem.solve(pulp.solvers.COIN_CMD())

# Print the solution!
for slot in range(slots):
    print(f"Slot {slot}:")
    for role in roles:
        for person in people:
            if pulp.value(assignments[slot, person, role]) == 1:
                print(f"    {role}: {person}")
                


PulpSolverError: Pulp: cannot execute cbc cwd: /Users/nightranger/workspace/datathon_c17

In [7]:
pulp.solvers()

TypeError: 'module' object is not callable